## 1. 라이브러리 및 데이터 불러오기

In [3]:
import pandas as pd

In [164]:
kosdaq = pd.read_excel("C:/Users/hanah/Desktop/workspace/Project/ESG/crawling/dart/kosdaq_report_final.xlsx")
ind_cd = pd.read_excel('한국표준산업분류(10차).xlsx')

## 2. 기업개요 중 업종코드 전처리

In [171]:
company_info = kosdaq.iloc[:,:13]
company_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1388 entries, 0 to 1387
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   고유번호     1388 non-null   int64         
 1   사업자등록번호  1388 non-null   object        
 2   종목코드     1388 non-null   int64         
 3   회사명      1388 non-null   object        
 4   업종       1388 non-null   object        
 5   주요제품     1385 non-null   object        
 6   상장일      1388 non-null   datetime64[ns]
 7   대표자명     1388 non-null   object        
 8   홈페이지     1324 non-null   object        
 9   지역       1388 non-null   object        
 10  주소       1388 non-null   object        
 11  업종코드     1388 non-null   int64         
 12  설립일      1388 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(8)
memory usage: 141.1+ KB


In [159]:
company_info['업종코드'] = company_info['업종코드'].astype('str')
company_info['ind_code'] = company_info['업종코드'].str[:2]

In [160]:
ind_dict = dict([(str(idx), value) for idx, value in zip(ind_cd['코드'], ind_cd['항목명'])])
for idx, code in enumerate(company_info['ind_code']):
    company_info.loc[idx, 'ind_code'] = ind_dict[code]

In [161]:
company_info['ind_code'].value_counts()

제조업                         918
정보통신업                       205
도매 및 소매업                     96
전문, 과학 및 기술 서비스업             70
금융 및 보험업                     39
건설업                          28
사업시설 관리, 사업 지원 및 임대 서비스업     12
교육 서비스업                       8
운수 및 창고업                      6
예술, 스포츠 및 여가관련 서비스업           5
숙박 및 음식점업                     3
Name: ind_code, dtype: int64

In [162]:
company_info.drop(['고유번호', '업종', '홈페이지', '지역', '업종코드', '상장일'], axis=1, inplace=True)
company_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1390 entries, 0 to 1389
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   사업자등록번호   1390 non-null   object
 1   종목코드      1390 non-null   int64 
 2   회사명       1390 non-null   object
 3   주요제품      1387 non-null   object
 4   대표자명      1390 non-null   object
 5   주소        1390 non-null   object
 6   설립일       1390 non-null   int64 
 7   ind_code  1390 non-null   object
dtypes: int64(2), object(6)
memory usage: 87.0+ KB


In [163]:
company_info.to_excel('company_info.xlsx', index=False)

## 3. 공시정보 전처리

In [198]:
temp1 = kosdaq.iloc[:,13:29]
temp1.insert(0, '결산연도', 2021)
temp1.columns = ['결산연도', '최대주주_성명', '최대주주_관계', '최대주주_주식종류', '최대주주_기초주식수', '최대주주_기초주식지분율', 
                 '최대주주_기말주식수', '최대주주_기말주식지분율', '소액주주수', '주주수', '소액주주비율', '소액주주_주식수',
                 '총주식수', '소액주주_주식지분율', '감사의견', '감사의견_강조사항', '핵심감사사항']

temp2 = kosdaq.iloc[:,29:]
temp2.insert(0, '결산연도', 2020)
temp2.columns = temp1.columns 
temp = pd.concat([temp1, temp2],ignore_index=True)

In [201]:
dart = pd.concat([kosdaq['회사명'], temp], axis=1)
dart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2776 entries, 0 to 2775
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   회사명           1388 non-null   object
 1   결산연도          2776 non-null   int64 
 2   최대주주_성명       2770 non-null   object
 3   최대주주_관계       2770 non-null   object
 4   최대주주_주식종류     2770 non-null   object
 5   최대주주_기초주식수    2770 non-null   object
 6   최대주주_기초주식지분율  2770 non-null   object
 7   최대주주_기말주식수    2770 non-null   object
 8   최대주주_기말주식지분율  2770 non-null   object
 9   소액주주수         2770 non-null   object
 10  주주수           2770 non-null   object
 11  소액주주비율        2770 non-null   object
 12  소액주주_주식수      2770 non-null   object
 13  총주식수          2770 non-null   object
 14  소액주주_주식지분율    2770 non-null   object
 15  감사의견          2770 non-null   object
 16  감사의견_강조사항     2770 non-null   object
 17  핵심감사사항        2770 non-null   object
dtypes: int64(1), object(17)
memory usage: 390.5+ KB


## 4. 소액주주 지분율(%) 실수화

In [213]:
dart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2764 entries, 0 to 2763
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CmpID             2764 non-null   int64  
 1   Year              2764 non-null   int64  
 2   MajorSHName       2764 non-null   object 
 3   MajorSHRelat      2764 non-null   object 
 4   MajorSHType       2764 non-null   object 
 5   MajorSHStrtNum    2764 non-null   object 
 6   MajorSHStrtRatio  2764 non-null   object 
 7   MajorSHEndNum     2764 non-null   object 
 8   MajorSHEndRatio   2764 non-null   object 
 9   MinorSHCnt        2764 non-null   int32  
 10  TotalSHCnt        2764 non-null   int32  
 11  MinorSHCntRatio   2764 non-null   float64
 12  MinorSHNum        2764 non-null   int32  
 13  TotalSHNum        2764 non-null   int32  
 14  MinorSHRatio      2764 non-null   float64
 15  AdtOpinion        2764 non-null   object 
 16  EmphsMatter       2764 non-null   object 


In [192]:
dart[['MinorSHCnt', 'TotalSHCnt', 'MinorSHCntRatio', 'MinorSHNum', 'TotalSHNum', 'MinorSHRatio']] = dart[['MinorSHCnt', 'TotalSHCnt', 'MinorSHCntRatio', 'MinorSHNum', 'TotalSHNum', 'MinorSHRatio']].replace('%', '', regex=True)
dart[['MinorSHCnt', 'TotalSHCnt', 'MinorSHCntRatio', 'MinorSHNum', 'TotalSHNum', 'MinorSHRatio']] = dart[['MinorSHCnt', 'TotalSHCnt', 'MinorSHCntRatio', 'MinorSHNum', 'TotalSHNum', 'MinorSHRatio']].replace('', 0)

dart[['MinorSHCnt', 'TotalSHCnt', 'MinorSHNum', 'TotalSHNum']] = dart[['MinorSHCnt', 'TotalSHCnt', 'MinorSHNum', 'TotalSHNum']].astype('int')
dart[['MinorSHCntRatio', 'MinorSHRatio']] = dart[['MinorSHCntRatio', 'MinorSHRatio']].astype('float')

## 5. 최대주주 우선주 제외 후 비율 총합 계산

In [ ]:
# temp = dart['MajorSHType'].str.split().str[-1]
# temp.loc[temp != '보통주'] 

In [220]:
dart['MajorSHType'].str.split().str[-1].unique()

array(['보통주', '증권예탁증권(DR)', '주식', '보통주(DR)', '의결권있는주식', '의결권없는주식',
       '(의결권있는주식)', '보통주식', '우선주', '〃', '보좉주', '전환우선주', '주식)', '전환사채',
       '(의결권있음)', '기명,보통', '"', '보통부', '기명식보통주', '의결권유', '//', '보퉁주'],
      dtype=object)

In [7]:
dart.loc[dart['MajorSHType'].str.contains('우선주')]

,CmpID,Year,MajorSHName,MajorSHRelat,MajorSHType,MajorSHStrtNum,MajorSHStrtRatio,MajorSHEndNum,MajorSHEndRatio,MinorSHCnt,TotalSHCnt,MinorSHCntRatio,MinorSHNum,TotalSHNum,MinorSHRatio,AdtOpinion,EmphsMatter,CoreAdtMatter,MajorSHSum
26,27,2021,강성민 조대원 ㈜아이마켓코리아 ㈜아이마켓코리아 안성수 김신원 이현숙 이두형 이숙자,본인 임원(상근이사) 특수관계인 특수관계인 임원(비등기) 임원(상근이사) 특수관계인...,보통주 보통주 보통주 우선주 보통주 보통주 보통주 보통주 보통주,"492,810 203,850 150,000 0 145,530 136,220 46,9...",8.95 3.70 2.73 0.00 2.64 2.47 0.85 0.54 0.25,"985,620 407,700 300,000 446,987 291,060 272,44...",8.33 3.45 2.54 2.30 0.79 0.50 0.23,28896,28909,99.96,8354681,11826022,70.65,적정,-,종속기업 현금창출 단위에 대한 손상평가,24.38
113,114,2021,정 신 손자현 이만종 이숙자 정미경 정우창 정혜민 정혜주 한상화 윤흥식 김니콜라스영...,본인 친인척 친인척 친인척 친인척 자녀 자녀 자녀 친인척 임원 임원 배우자 임원 임...,보통주 보통주 보통주 보통주 보통주 보통주 보통주 보통주 보통주 보통주 보통주 보통...,"5,091,427 22,500 22,500 22,500 22,500 100 100 ...",26.91 0.12 0.12 0.12 0.12 0.00 0.00 0.00 0.00 ...,"6,584,640 33,750 33,750 33,750 33,750 150 150 ...",22.28 0.11 0.11 0.11 0.11 0.00 0.00 0.00 0.02 ...,16428,16438,99.94,18265518,29556038,61.80,적정,-,감가상각 개시시점의 적절성(주1),26.72
231,232,2021,시너지파트너스(주) 시너지파트너스(주) 시너지아이비투자(주) 시너지아이비투자(주) ...,최대주주 최대주주 최대주주 최대주주 특수관계인 특수관계인,우선주 보통주 우선주 보통주 우선주 보통주,"0 0 5,557,378 0 793,911 0",0.00 0.00 13.69 0.00 1.96 0.00,"0 2,000,000 0 1,111,475 0 158,782",10.99 1.57,14734,14741,99.95,6477001,10110899,64.06,의견거절,"-내부회계관리제도의 중요한 취약점,",-내부회계관리제도의 중요한 취약점,32.32
297,298,2021,APS홀딩스(주) APS홀딩스(주) 김영주 유석준 김혁,최대주주 본인 최대주주 본인 발행회사임원 특수관계인 특수관계인,보통주 우선주 보통주 보통주 보통주,"3,019,175 801,194 2,500 5,940 555",20.85 0.00 0.02 0.04 0.00,"3,820,369 0 2,500 5,940 555",25.00 0.04,50913,50919,99.90,10719308,15281421,70.10,-,-,-,25.06
348,349,2021,(주)폴라리스오피스 (주)폴라리스오피스 에스와이미디어그룹(주) 조성우 지준경 이성엽...,최대주주 최대주주 특수관계인 특수관계인 특수관계인 특수관계인 특수관계인,보통주 우선주 보통주 보통주 보통주 보통주 보통주,"6,211,180 0 0 78,860 0 0 0",12.54 0.00 0.00 0.16 0.00 0.00 0.00,"6,211,180 157,130 1,237,065 1,049,733 213,695 ...",10.77 1.82 0.37 0.02 0.02,5979,5984,99.92,42122937,56509565,74.54,적정,해당사항 없음,1)금융자산의평가\n2)유형자산손상검토,15.41
373,374,2021,구자옥 구자옥,최대주주본인 최대주주본인,보통주 우선주,"4,706,794 0",40.10 0.00,"4,706,794 689,607",40.10,1926,1936,99.48,4259911,11723182,36.34,적정,-,"종속기업투자, \n현금창출단위손상",46.00
415,416,2021,심텍홀딩스 박인은 박정권 서상덕 지석래 심텍홀딩스 김영구 김연호 김갑수 이경수 전원...,최대주주 계열회사 임원 계열회사 임원 계열회사 임원 계열회사 임원 최대주주 계열회사...,보통주 보통주 보통주 보통주 보통주 우선주 보통주 보통주 보통주 보통주 보통주 보통...,"11,277,150 20,569 0 0 0 710,660 2,092 5,501 25...",35.40 0.10 0.00 0.00 0.00 100.00 0.00 0.00 0.1...,"10,527,150 20,569 5,097 10,000 6,000 0 2,092 5...",33.00 0.10 0.10,49411,49427,99.96,18505141,31845877,58.10,"""적정""","""해당사항없음""",국외매출과 관련된 수익의 기간귀속,33.20
485,486,2021,이상제일호 사모투자 합자회사 이상제일호 사모투자 합자회사 왕훈식 박준상 박재경 심지...,최대주주 최대주주 대표이사 임원 임원 임원 감사 임원,우선주 보통주 보통주 보통주 보통주 보통주 보통주 보통주,"5,016,722 0 1,867,734 1,947,274 356,833 152,08...",9.61 0.00 3.58 3.73 0.68 0.29 0.14 0.00,"0 9,967,217 1,889,520 1,896,326 420,422 218,93...",16.11 3.05 3.07 0.68 0.35 0.12 0.20,13788,13796,99.94,40349133,61854061,65.23,적정,-,-,23.58
528,529,2021,에이아이트리 유한회사 에이아이트리 유한회사 김기문 박용준 최병철 박종후 테스나,최대주주 최대주주 등기임원 미등기임원 미등기임원 미등기임원 자사주,보통주 우선주 보통주 보통주 보통주 보통주 보통주,"2,909,292 2,320,672 15,000 7,000 39,200 2,500 ...",17.03 13.59 0.09 0.04 0.23 0.01 0.52,"2,909,292 2,320,672 5,000 7,000 39,200 2,500 8...",17.03 0.03 0.04 0.23 0.01 0.52,54120,54130,99.97,8955921,17080249,52.43,적정,-,-,31.45
593,594,2021,(주)초록뱀미디어 (주)초록뱀미디어,최대주주 최대주주,보통주 우선주,"2,064,220 2,412,280",4.79 5.60,"2,663,022 2,412,280",6.09,21526,43693485,100.00,21521,35989189,82.37,적정,"(*1),(*2)",건설중인자산의 실재성 및 손상검토,11.61


In [420]:
dart['MajorSHType'] = dart['MajorSHType'].str.replace('전환', ' ').str.replace('\n', '', regex=True)

In [421]:
index = dart.loc[dart['MajorSHType'].str.contains('우선주')].index
print(index)

for i in index:
    idx = dart.loc[i, 'MajorSHType'].split().index('우선주')
    target = dart.loc[i, 'MajorSHEndRatio'].split()
    dart.loc[i, 'MajorSHEndRatio'] = dart.loc[i, 'MajorSHEndRatio'].replace(target[idx], '') 

Int64Index([  26,  113,  231,  297,  348,  373,  415,  485,  528,  593,  654,
             782, 1034, 1185, 1200, 1239, 1243, 1385, 1390, 1442, 1495, 1545,
            1613, 1679, 1797, 1867, 1910, 2036, 2164, 2561, 2567, 2625],
           dtype='int64')


In [424]:
temp = dart['MajorSHEndRatio'].str.split(expand=True)
temp.fillna(0, inplace=True)
temp = temp.astype('float')
dart['MajorSHSum'] = temp.sum(axis=1)

In [428]:
dart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2764 entries, 0 to 2763
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CmpID             2764 non-null   int64  
 1   Year              2764 non-null   int64  
 2   MajorSHName       2764 non-null   object 
 3   MajorSHRelat      2764 non-null   object 
 4   MajorSHType       2764 non-null   object 
 5   MajorSHStrtNum    2764 non-null   object 
 6   MajorSHStrtRatio  2764 non-null   object 
 7   MajorSHEndNum     2764 non-null   object 
 8   MajorSHEndRatio   2764 non-null   object 
 9   MinorSHCnt        2764 non-null   int64  
 10  TotalSHCnt        2764 non-null   int64  
 11  MinorSHCntRatio   2764 non-null   float64
 12  MinorSHNum        2764 non-null   int64  
 13  TotalSHNum        2764 non-null   int64  
 14  MinorSHRatio      2764 non-null   float64
 15  AdtOpinion        2764 non-null   object 
 16  EmphsMatter       2764 non-null   object 


In [10]:
dart.to_excel('dart_info.xlsx', index=False)